# Convolutional Neuronal Network (CNN)

In [1]:
import logging
from os import environ
logging.disable(logging.WARNING)  # Suppress deprecation warnings

Primero se importan todas las funciones a utilizar de la librería Keras.

In [2]:
import numpy as np
import time
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


Ahora se definen algunos parámetros que se utilizan en el preprocesado como el nuevo tamaño de las imágenes y la normalización de los píxeles.

In [3]:
#Model params(
img_rows   = 80  # Training data size is 480
img_cols   = 80  # Training data size is 480
rescale    = 1./255
batch_size = 10
num_epoch  = 6
classes    = ['d4', 'd6', 'd8', 'd10', 'd12', 'd20']
directory  = './dice/'

Con los parámetros que se definieron se realiza el preprocesado utilizando la función ImageDataGenerator.

In [4]:
print('Data preprocessing')
input_shape = (img_rows, img_cols, 1)
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    directory   = directory + './train/',
    target_size = (img_rows, img_cols),
    classes     = classes,
    color_mode  = 'grayscale',
    batch_size  = batch_size,
)
        
valid_generator = ImageDataGenerator(rescale=1./255,).flow_from_directory(
    directory   = directory + './valid/',
    target_size = (img_rows, img_cols),
    classes     = classes,
    color_mode  = 'grayscale',
    batch_size  = batch_size,
)

Data preprocessing
Found 14284 images belonging to 6 classes.
Found 2102 images belonging to 6 classes.


Ahora se procede a construir el modelo de aprendizaje.

In [5]:
# Model design
model = Sequential()

# Adding convolution and pooling layers
model.add(Conv2D(8,kernel_size=(3,3),activation='relu',input_shape=input_shape))
model.add(Conv2D(16,kernel_size=(3,3),activation='relu'))
model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Randomly turn on and off some neurons to improve convergence
model.add(Dropout(0.25))

# Adding flatten layer
model.add(Flatten())

# Adding fully connected layer
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))

# Adding softmax layer for final categorization
model.add(Dense(len(classes),activation='softmax'))

# Compile model with Adadelta optimizer
model.compile(loss=categorical_crossentropy,optimizer=Adadelta(),metrics=['accuracy'])

# Model summary:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 78, 78, 8)         80        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 76, 76, 16)        1168      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 74, 74, 32)        4640      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 36, 36, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 82944)             0         
__________

Solo falta entrenar el modelo, se van a utilizar 6 iteraciones.

In [6]:
# Model training
start = time.time()

model_hist = model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epoch,
    verbose=1,
    validation_data=valid_generator,
    validation_steps=len(valid_generator)
)
        
end = time.time()
print("Training time: {:.0f}s".format(end-start))

Epoch 1/6
1429/1429 [==============================] - 286s 200ms/step - loss: 1.2126 - acc: 0.5131 - val_loss: 0.6177 - val_acc: 0.8092
Epoch 2/6
1429/1429 [==============================] - 295s 206ms/step - loss: 0.5858 - acc: 0.7861 - val_loss: 0.3815 - val_acc: 0.9144
Epoch 3/6
1429/1429 [==============================] - 280s 196ms/step - loss: 0.3729 - acc: 0.8636 - val_loss: 0.2858 - val_acc: 0.9477
Epoch 4/6
1429/1429 [==============================] - 332s 232ms/step - loss: 0.2789 - acc: 0.8992 - val_loss: 0.3353 - val_acc: 0.9320
Epoch 5/6
1429/1429 [==============================] - 366s 256ms/step - loss: 0.2468 - acc: 0.9100 - val_loss: 0.2975 - val_acc: 0.9405
Epoch 6/6
1429/1429 [==============================] - 280s 196ms/step - loss: 0.2266 - acc: 0.9220 - val_loss: 0.2609 - val_acc: 0.9567
Training time: 1840s


Se puede ver que en la última iteración se alcanzó una precisión de un 95,67% en el set de validación.

El siguiente paso el evaluar la red utilizando el set de prueba.

In [48]:
# Model results on the ./test dataset:
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    directory   ='./test/',
    target_size =(img_rows, img_cols),
    classes     =classes,
    color_mode  ='grayscale',
    batch_size  =batch_size
)
    
model_results = model.evaluate_generator(test_generator, steps=len(test_generator))
model_results

for idx, label in enumerate(model.metrics_names):
    print('{}: {}'.format(label, model_results[idx]))

Found 40 images belonging to 6 classes.
loss: 0.00357031896806
acc: 1.0


Por último se puede ver como en el set de prueba se acertaron todas las etiquetas de las imágenes por lo que se concluye que se logró un modelo de aprendizaje que generaliza muy bien.